# DISEÑO DEL PROYECTO

Este caso simula que somos una empresa inmobiliaria que hace inversiones en grandes y medianas ciudades comprando inmuebles para posteriormente alquilarlos como apartamentos turísticos.

La dirección ha tomado la decisión de invertir en Sevilla, y nos ha encargado analizar los datos que el líder del sector AirBnb hace públicos para intentar encontrar los tipos de inmuebles que tienen mayor potencial comercial para alquiler turístico. Desean también tener un sistema de predicción de la ocupación que tendrían las viviendas si las comprarán para hacer así una estimación de las ganancias que obtendrían.

Como entregable principal esperan la tipología (o tipologías) de inmuebles que el equipo de valoraciones debe buscar entre las oportunidades existentes en la ciudad y los principales barrios o zonas geográficas en las que focalizarse. Como bonus, se le propone una herramienta donde introduciendo ciertas características del inmueble, se devuelva la ganancia esperada por el alquiler del mismo.

Para cumplir con el objetivo aplicaremos la metodología de Discovery y las técnicas de BA aprendidas así como las técnicas de modelado en ML.

# SET UP

## CREAR EL ENTORNO PARA EL PROYECTO

## IMPORTAR PAQUETES

Este primer paquete "os" sistema operativo en español, sirve para crear la estructura de los directorios. Es un paquete de Python y permite interactuar con el sistema operativo para crear las carpetas.

In [1]:
import os
import numpy as np
import pandas as pd

#Automcompletar rápido
%config IPCompleter.greedy=True

## CREAR EL DIRECTORIO DEL PROYECTO

### Definir el directorio raiz donde vamos a crear el proyecto

In [2]:
raiz = 'C:/Users/tania.camacho/Desktop/Master/EstructuraDirectorio/04_PORTAFOLIO/'

### Definir el nombre del proyecto

In [3]:
nombre_dir = '01_PORTAFOLIO_INMO'

### Crear el directorio y la estructura del proyecto

**IMPORTANTE**: La creación de estas carpetas no sobreescriben si ya existieran.

In [4]:
path = raiz + nombre_dir

In [9]:
path = raiz + nombre_dir

try:
    os.mkdir(path)
    os.mkdir(path + '/01_Documentos')
    os.mkdir(path + '/02_Datos')
    os.mkdir(path + '/02_Datos/01_Originales')
    os.mkdir(path + '/02_Datos/02_Validacion')
    os.mkdir(path + '/02_Datos/03_Trabajo')
    os.mkdir(path + '/02_Datos/04_Caches')
    os.mkdir(path + '/03_Notebooks')
    os.mkdir(path + '/03_Notebooks/01_Funciones')
    os.mkdir(path + '/03_Notebooks/02_Desarrollo')
    os.mkdir(path + '/03_Notebooks/03_Sistema')
    os.mkdir(path + '/04_Modelos')
    os.mkdir(path + '/05_Resultados')
    os.mkdir(path + '/09_Otros')
    
except OSError:
    print ("La creación del directorio %s ha fallado" % path)
else:
    print ("Se ha creado satisfactoriamente el directorio %s " % path)

La creación del directorio C:/Users/tania.camacho/Desktop/Master/EstructuraDirectorio/04_PORTAFOLIO/01_PORTAFOLIO_INMO ha fallado


Explicación de las carpetas creadas y para que sirven:
- 01 Documentos: para documentos que nos aporte el cliente y guardar el documento yml del entorno.
- 02 Datos: donde se almacenan los datos recibidos. Se subdividen en:
    - Datos originales: lo que se recibe original del cliente.
    - Datos de validación: sobre los originales extraemos una muestra para la validacion o test final y se queda aquí guardado.
    - Datos trabajo: que son los datos sobre los que se hace el desarrollo.
    - Datos caché: Carpeta para ir guardando los avances cuando termina el dia de trabajo sin tener que estar lanzando cada proceso porque puede haber algunos que tarde horas.
- 03 Notebooks: que se subdivide a su vez en 3 más:
    - Funciones, que es el NB para guardar las funciones usadas y que se necesitan.
    - El de desarrollo de código
    - El de sistema para guardar los notebook que finalmente vayan a producción.
- 04 Modelos: Para guardar los modelos generados y el modelo definitivo
- 05 Resultados: Para guardar csv o excel con resultados de algunos análisis, insights encontrados, etc.
- 06 Otros: cajón de sastre.

### Situar el directorio de trabajo en el proyecto

In [10]:
os.chdir(path)

## CREAR LOS DATASETS INICIALES

Poner los datos originales en la carpeta '/02_Datos/01_Originales'

### Importar datos originales

Usaremos datos reales ofrecidos por AirBnB en esta página: http://insideairbnb.com/get-the-data.html

Como ves AirBnb entrega los datos en formato csv y con todo en inglés. Vamos a quedarnos sólo con las tablas que nos vayan a resultar útiles que son la de listings y listings_det.

#### Fichero listings

In [5]:
nombre_fichero_datos = 'listings.csv'

Cargar los datos.

In [7]:
ruta_completa = path + '/02_Datos/01_Originales/' + nombre_fichero_datos

listings = pd.read_csv(ruta_completa)
listings.head(5)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,702998,"NICE FLAT, 2 TV's AND FREE WIFI",3609455,Javier,Casco Antiguo,Santa Catalina,37.39080,-5.98543,Entire home/apt,59.0,3,358,2024-03-15,4.57,1,0,64,CTC-2018127385
1,703547,Barrio de Triana. Auténtico y tradicional,166618468,Javier,Triana,Triana Oeste,37.38603,-6.01132,Entire home/apt,154.0,2,48,2024-03-17,0.34,22,252,2,VFT/SE/00936
2,704131,Reservaloen Casa del Museo studio 2,3258098,Apartamentos Reservaloen,Casco Antiguo,Museo,37.39164,-6.00022,Entire home/apt,253.0,1,1,2016-10-15,0.01,22,119,0,A/SE/000167
3,705652,Historic home downtown Seville1,3625249,Lola,Casco Antiguo,San Vicente,37.39635,-5.99868,Private room,42.0,2,447,2023-10-29,3.20,1,307,24,VFT/SE/00414
4,705697,Room near center +breakfast,3625534,Irina,Triana,Triana Este,37.37763,-6.00640,Private room,35.0,3,338,2024-03-21,2.52,1,190,43,VFT/SE/04584


#### Fichero listings_det

In [8]:
nombre_fichero_datos = 'listings_det.csv'

In [9]:
ruta_completa = path + '/02_Datos/01_Originales/' + nombre_fichero_datos

listings_det = pd.read_csv(ruta_completa)
listings_det.head(5)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,702998,https://www.airbnb.com/rooms/702998,20240331032628,2024-03-31,city scrape,"NICE FLAT, 2 TV's AND FREE WIFI","Gorgeous flat in the centre of Seville, 5 minu...","The sunniest neighbourhood of Sevilla, and par...",https://a0.muscache.com/pictures/ebf5b2ca-0fcf...,3609455,...,4.87,4.79,4.53,CTC-2018127385,t,1,1,0,0,4.57
1,703547,https://www.airbnb.com/rooms/703547,20240331032628,2024-03-31,city scrape,Barrio de Triana. Auténtico y tradicional,My house is your house!<br /><br />The apartme...,NaN,https://a0.muscache.com/pictures/695f4468-95ec...,166618468,...,4.76,4.31,4.22,VFT/SE/00936,f,22,16,6,0,0.34
2,704131,https://www.airbnb.com/rooms/704131,20240331032628,2024-03-31,city scrape,Reservaloen Casa del Museo studio 2,Apartamentos Casa del Museo is located next to...,NaN,https://a0.muscache.com/pictures/4c911f2d-2932...,3258098,...,5.00,5.00,5.00,A/SE/000167,t,22,22,0,0,0.01
3,705652,https://www.airbnb.com/rooms/705652,20240331032628,2024-03-31,city scrape,Historic home downtown Seville1,NaN,NaN,https://a0.muscache.com/pictures/10883694/37d2...,3625249,...,4.66,4.77,4.57,VFT/SE/00414,f,1,0,1,0,3.20
4,705697,https://www.airbnb.com/rooms/705697,20240331032628,2024-03-31,city scrape,Room near center +breakfast,Private double room in TRIANA. 15 min distanc...,There are the buses and underground stops near...,https://a0.muscache.com/pictures/9866974/b8d91...,3625534,...,4.98,4.77,4.92,VFT/SE/04584,t,1,0,1,0,2.52


Como hay demasiadas variables en este df vamos a quedarnos solo con las que consideramos más interesantes.

In [10]:
a_incluir = ['id',
             'accommodates',
             'bathrooms',
             'bedrooms',
             'beds',
             'description']
listings_det = listings_det.loc[:,a_incluir]
listings_det

,id,accommodates,bathrooms,bedrooms,beds,description
0,702998,2,1.0,1.0,1.0,"Gorgeous flat in the centre of Seville, 5 minu..."
1,703547,4,1.0,2.0,2.0,My house is your house!<br /><br />The apartme...
2,704131,4,1.0,1.0,2.0,Apartamentos Casa del Museo is located next to...
3,705652,2,1.0,1.0,2.0,NaN
4,705697,1,1.0,1.0,1.0,Private double room in TRIANA. 15 min distanc...
...,...,...,...,...,...,...
7708,1122218429501569213,4,1.0,1.0,2.0,"Relax and unwind in this quiet, fully renovate..."
7709,1122329926019227935,2,2.0,2.0,3.0,Forget the worries in this great home - it's a...
7710,1122475005140746995,4,2.0,1.0,2.0,This luxury apartment in Seville has recently ...
7711,1122504379533066462,1,2.0,1.0,1.0,Stay in this extraordinary home and don't miss...


#### Fichero idealista

En nuestros datos no tenemos el precio de compra de un inmueble, pero consideramos que es un dato importante para conocer las predicciones de ganancias. Por tanto, vamos a buscar esos datos de forma externa.

En esta página tenemos justo la info que necesitamos: https://www.idealista.com/sala-de-prensa/informes-precio-vivienda/venta/andalucia/sevilla-provincia/sevilla/

Podemos extraerla de forma sencilla con el plugin instant data scraper de Chrome, y guardarla en nuestra carpeta Datos con el nombre 'precio_m2.csv'

In [11]:
nombre_fichero_datos = 'precio_m2.csv'

In [12]:
ruta_completa = path + '/02_Datos/01_Originales/' + nombre_fichero_datos

precio_m2 = pd.read_csv(ruta_completa) \
    .loc[1:,['table__cell 2','icon-elbow']] \
    .rename(columns = {'table__cell 2':'precio_m2','icon-elbow':'distrito'})
precio_m2

,precio_m2,distrito
1,2.008 €/m2,Bellavista - Jardines de Hércules
2,3.601 €/m2,Centro
3,1.218 €/m2,Cerro Amate
4,2.814 €/m2,La Palmera - Los Bermejales
5,3.145 €/m2,Los Remedios
6,1.671 €/m2,Macarena
7,2.953 €/m2,Nervión
8,1.389 €/m2,Parque Alcosa
9,1.398 €/m2,Pino Montano
10,3.294 €/m2,Prado de San Sebastián - Felipe II - Bueno Mon...


In [13]:
#Vamos a separar el texo del numero para que asi no tengamos el €/m2
precio_m2['precio_m2'] = precio_m2.precio_m2.str.split(expand= True)[0].str.replace('.','', regex = False).astype('int')
precio_m2

,precio_m2,distrito
1,2008,Bellavista - Jardines de Hércules
2,3601,Centro
3,1218,Cerro Amate
4,2814,La Palmera - Los Bermejales
5,3145,Los Remedios
6,1671,Macarena
7,2953,Nervión
8,1389,Parque Alcosa
9,1398,Pino Montano
10,3294,Prado de San Sebastián - Felipe II - Bueno Mon...


### Creación tablón analítico

#### Union ficheros listings y listings_det

In [14]:
df = pd.merge(left = listings, right = listings_det, how = 'left', on = 'id')
df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,accommodates,bathrooms,bedrooms,beds,description
0,702998,"NICE FLAT, 2 TV's AND FREE WIFI",3609455,Javier,Casco Antiguo,Santa Catalina,37.390800,-5.985430,Entire home/apt,59.0,...,4.57,1,0,64,CTC-2018127385,2,1.0,1.0,1.0,"Gorgeous flat in the centre of Seville, 5 minu..."
1,703547,Barrio de Triana. Auténtico y tradicional,166618468,Javier,Triana,Triana Oeste,37.386030,-6.011320,Entire home/apt,154.0,...,0.34,22,252,2,VFT/SE/00936,4,1.0,2.0,2.0,My house is your house!<br /><br />The apartme...
2,704131,Reservaloen Casa del Museo studio 2,3258098,Apartamentos Reservaloen,Casco Antiguo,Museo,37.391640,-6.000220,Entire home/apt,253.0,...,0.01,22,119,0,A/SE/000167,4,1.0,1.0,2.0,Apartamentos Casa del Museo is located next to...
3,705652,Historic home downtown Seville1,3625249,Lola,Casco Antiguo,San Vicente,37.396350,-5.998680,Private room,42.0,...,3.20,1,307,24,VFT/SE/00414,2,1.0,1.0,2.0,NaN
4,705697,Room near center +breakfast,3625534,Irina,Triana,Triana Este,37.377630,-6.006400,Private room,35.0,...,2.52,1,190,43,VFT/SE/04584,1,1.0,1.0,1.0,Private double room in TRIANA. 15 min distanc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7708,1122218429501569213,Tartessos Apartamentos 1,190448586,Gualberto,San Pablo - Santa Justa,"San Carlos, Tartessos",37.398657,-5.970950,Entire home/apt,137.0,...,NaN,2,113,0,VUT/SE/12948,4,1.0,1.0,2.0,"Relax and unwind in this quiet, fully renovate..."
7709,1122329926019227935,piso acogedor y tranquilo,68601310,Lola,San Pablo - Santa Justa,"Carretera de Carmona, María Auxiliadora, Fontanal",37.397021,-5.978117,Private room,42.0,...,NaN,1,252,0,Exempt,2,2.0,2.0,3.0,Forget the worries in this great home - it's a...
7710,1122475005140746995,EDomus Setas Parking Boutique,525452885,Jorge,Casco Antiguo,Santa Catalina,37.392670,-5.988250,Entire home/apt,116.0,...,NaN,11,267,0,VFT/SE/05265,4,2.0,1.0,2.0,This luxury apartment in Seville has recently ...
7711,1122504379533066462,habitación individual sin baño,537676707,Pension,Casco Antiguo,San Bartolomé,37.385839,-5.986283,Private room,90.0,...,NaN,11,269,0,H/SE/00889,1,2.0,1.0,1.0,Stay in this extraordinary home and don't miss...


#### Union df con precio_m2

In [15]:
#Analizamos los distritos que tenemos en el df de precio_m2
precio_m2.distrito.unique()

array(['Bellavista - Jardines de Hércules', 'Centro', 'Cerro Amate',
       'La Palmera - Los Bermejales', 'Los Remedios', 'Macarena',
       'Nervión', 'Parque Alcosa', 'Pino Montano',
       'Prado de San Sebastián - Felipe II - Bueno Monreal',
       'San Jerónimo', 'Santa Clara',
       'Santa Justa - Miraflores - Cruz Roja', 'Sevilla este',
       'Torreblanca', 'Triana'], dtype=object)

In [16]:
#Analizamos los distritos que tenemos en el df
df.neighbourhood_group.unique()

array(['Casco Antiguo', 'Triana', 'Macarena', 'Nervión', 'Cerro - Amate',
       'Sur', 'Palmera - Bellavista', 'San Pablo - Santa Justa',
       'Los Remedios', 'Este - Alcosa - Torreblanca', 'Macarena - Norte'],
      dtype=object)

Como existen diferentes nombres de distritos vamos a hacer un cambio en precio_m2 para que tengan los mismos nombres y así poder cruzar. Las asignaciones van a ser:
- Centro --> Casco Antiguo
- Cerro Amate --> Cerro - Amate
- Prado de San Sebastián - Felipe II - Bueno Monreal --> Sur
- La Palmera - Los Bermejales --> Palmera - Bellavista
- Santa Justa - Miraflores - Cruz Roja --> San Pablo - Santa Justa
- Sevilla este --> Este - Alcosa - Torreblanca
- Pino Montano --> Macarena - Norte

El resto de distritos quedan igual y no se harán cambios:
- Triana
- Macarena
- Nervión
- Los Remedios

In [17]:
precio_m2.distrito = precio_m2.distrito.replace({'Centro':'Casco Antiguo',
                                                 'Cerro Amate':'Cerro - Amate',
                                                 'Prado de San Sebastián - Felipe II - Bueno Monreal':'Sur',
                                                 'La Palmera - Los Bermejales':'Palmera - Bellavista',
                                                 'Santa Justa - Miraflores - Cruz Roja':'San Pablo - Santa Justa',
                                                 'Sevilla este' : 'Este - Alcosa - Torreblanca',
                                                 'Pino Montano':'Macarena - Norte',})
precio_m2

,precio_m2,distrito
1,2008,Bellavista - Jardines de Hércules
2,3601,Casco Antiguo
3,1218,Cerro - Amate
4,2814,Palmera - Bellavista
5,3145,Los Remedios
6,1671,Macarena
7,2953,Nervión
8,1389,Parque Alcosa
9,1398,Macarena - Norte
10,3294,Sur


Creamos el datamart final.

In [18]:
datos = pd.merge(left = df, right = precio_m2, how='left', left_on= 'neighbourhood_group', right_on='distrito')
datos

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,availability_365,number_of_reviews_ltm,license,accommodates,bathrooms,bedrooms,beds,description,precio_m2,distrito
0,702998,"NICE FLAT, 2 TV's AND FREE WIFI",3609455,Javier,Casco Antiguo,Santa Catalina,37.390800,-5.985430,Entire home/apt,59.0,...,0,64,CTC-2018127385,2,1.0,1.0,1.0,"Gorgeous flat in the centre of Seville, 5 minu...",3601,Casco Antiguo
1,703547,Barrio de Triana. Auténtico y tradicional,166618468,Javier,Triana,Triana Oeste,37.386030,-6.011320,Entire home/apt,154.0,...,252,2,VFT/SE/00936,4,1.0,2.0,2.0,My house is your house!<br /><br />The apartme...,3118,Triana
2,704131,Reservaloen Casa del Museo studio 2,3258098,Apartamentos Reservaloen,Casco Antiguo,Museo,37.391640,-6.000220,Entire home/apt,253.0,...,119,0,A/SE/000167,4,1.0,1.0,2.0,Apartamentos Casa del Museo is located next to...,3601,Casco Antiguo
3,705652,Historic home downtown Seville1,3625249,Lola,Casco Antiguo,San Vicente,37.396350,-5.998680,Private room,42.0,...,307,24,VFT/SE/00414,2,1.0,1.0,2.0,NaN,3601,Casco Antiguo
4,705697,Room near center +breakfast,3625534,Irina,Triana,Triana Este,37.377630,-6.006400,Private room,35.0,...,190,43,VFT/SE/04584,1,1.0,1.0,1.0,Private double room in TRIANA. 15 min distanc...,3118,Triana
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7708,1122218429501569213,Tartessos Apartamentos 1,190448586,Gualberto,San Pablo - Santa Justa,"San Carlos, Tartessos",37.398657,-5.970950,Entire home/apt,137.0,...,113,0,VUT/SE/12948,4,1.0,1.0,2.0,"Relax and unwind in this quiet, fully renovate...",2392,San Pablo - Santa Justa
7709,1122329926019227935,piso acogedor y tranquilo,68601310,Lola,San Pablo - Santa Justa,"Carretera de Carmona, María Auxiliadora, Fontanal",37.397021,-5.978117,Private room,42.0,...,252,0,Exempt,2,2.0,2.0,3.0,Forget the worries in this great home - it's a...,2392,San Pablo - Santa Justa
7710,1122475005140746995,EDomus Setas Parking Boutique,525452885,Jorge,Casco Antiguo,Santa Catalina,37.392670,-5.988250,Entire home/apt,116.0,...,267,0,VFT/SE/05265,4,2.0,1.0,2.0,This luxury apartment in Seville has recently ...,3601,Casco Antiguo
7711,1122504379533066462,habitación individual sin baño,537676707,Pension,Casco Antiguo,San Bartolomé,37.385839,-5.986283,Private room,90.0,...,269,0,H/SE/00889,1,2.0,1.0,1.0,Stay in this extraordinary home and don't miss...,3601,Casco Antiguo


In [19]:
#Comprobamos nulos
datos.precio_m2.isna().sum()

0

In [20]:
#Vemos info de df final
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7713 entries, 0 to 7712
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              7713 non-null   int64  
 1   name                            7713 non-null   object 
 2   host_id                         7713 non-null   int64  
 3   host_name                       7713 non-null   object 
 4   neighbourhood_group             7713 non-null   object 
 5   neighbourhood                   7713 non-null   object 
 6   latitude                        7713 non-null   float64
 7   longitude                       7713 non-null   float64
 8   room_type                       7713 non-null   object 
 9   price                           7417 non-null   float64
 10  minimum_nights                  7713 non-null   int64  
 11  number_of_reviews               7713 non-null   int64  
 12  last_review                     67

### Extraer y reservar el dataset de validación

El método sample() sirve para extraer una muestra. Con el parámetro frac, le decimos el % de la muestra que queremos que extraiga del original. Parece que es un método de Python base.

In [21]:
val= datos.sample(frac = 0.3)

In [22]:
nombre_fichero_validacion = 'validacion.csv'

ruta_completa = path + '/02_Datos/02_Validacion/' + nombre_fichero_validacion

val.to_csv(ruta_completa)

### Extraer y guardar el dataset de trabajo

In [23]:
trabajo = datos.loc[~datos.index.isin(val.index)]

In [24]:
nombre_fichero_trabajo = 'trabajo.csv'

ruta_completa = path + '/02_Datos/03_Trabajo/' + nombre_fichero_trabajo

trabajo.to_csv(ruta_completa)